In [1]:
import numpy as np
import pandas as pd

In [2]:
from energy_consumption.models.ec_model1 import get_ec_forecasts_model1

energy_forecasts_m1 = get_ec_forecasts_model1()

if 'date_time' in energy_forecasts_m1.columns:
    energy_forecasts_m1 = energy_forecasts_m1.drop(columns=['date_time'])

100%|██████████| 256/256 [02:57<00:00,  1.44it/s]


In [3]:
energy_forecasts_m1

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-13,energy,36 hour,48.929369,57.491332,59.923923,62.483252,70.495715
1,2023-11-13,energy,40 hour,49.559434,56.678865,57.988762,60.082022,67.115966
2,2023-11-13,energy,44 hour,39.901500,44.766501,46.955923,49.420344,56.955538
3,2023-11-13,energy,60 hour,48.929369,57.491332,59.923923,62.483252,70.495715
4,2023-11-13,energy,64 hour,49.559434,56.678865,57.988762,60.082022,67.115966
5,2023-11-13,energy,68 hour,39.901500,44.766501,46.955923,49.420344,56.955538


In [8]:
from dax.models.dax_model1 import get_dax_forecasts_model1
df_sub_dax = get_dax_forecasts_model1()

In [9]:
from datetime import datetime
date_st = (datetime.today().strftime('%Y-%m-%d'))

df_sub_dax.insert(0, 'forecast_date', date_st)

In [10]:
import pandas as pd 

weeks = ['0 week', '1 week', '2 week', '3 week', '4 week']

df_sub_inf = pd.DataFrame({
    "forecast_date": date_st,
    "target": "infections",
    "horizon": weeks,
    "q0.025": np.nan,
    "q0.25": np.nan,
    "q0.5": np.nan,
    "q0.75": np.nan,
    "q0.975": np.nan})

In [11]:

submission_frame = pd.merge(df_sub_dax, energy_forecasts_m1, how='outer')
submission_frame = pd.merge(submission_frame, df_sub_inf, how='outer')

In [12]:
date_string = date_st
date_obj = datetime.strptime(date_string, "%Y-%m-%d")
formatted_date = date_obj.strftime("%Y%m%d")

submission_frame.to_csv(
    str('C:/Users/Maria/Documents/Studium/Pyhton Projekte/PTSFC/submission_files/' +
        formatted_date + '_BrienneVonTarth'),
    index=False, na_rep="NA")